In [ ]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V5

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V6

In [3]:
!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V7

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
graphnet: INFO     2023-02-06 20:30:14 - get_logger - Writing log to logs/graphnet_20230206-203014.log
Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_16', 'MODEL_NAME': <class 'icecube.models.EncoderWithDirectionReconstruction'>, 'LOSS_FUNC': <class 'graphnet.training.loss_functions.VonMisesFisher3DLoss'>, 'METRIC': <function get_score_vector at 0x7f231d4a4b00>, 'TRN_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV8'>, 'VAL_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV8'>, 'MODEL_WTS': False, '__doc__': None}
epoch     train_loss  valid_loss  val_metric
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 0 with value: 1.149692177772522.           
   epoch  train_loss  valid_loss     metric
0      0    2.025684    1.899114  1.1496922
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
Better model found at epoch 1 with value: 1.1020729541778564.          
   epoch  train_loss  valid_loss    metric
0      1    1.847046    1.777812  1.102073
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 2 with value: 1.0806740522384644.          
   epoch  train_loss  valid_loss    metric
0      2    1.735712    1.692183  1.080674
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
Better model found at epoch 3 with value: 1.064226508140564.           
   epoch  train_loss  valid_loss     metric
0      3    1.676948    1.652733  1.0642265
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
Better model found at epoch 4 with value: 1.0579525232315063.          
   epoch  train_loss  valid_loss  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 5 with value: 1.0564767122268677.    
   epoch  train_loss  valid_loss     metric
0      5    1.622458    1.612727  1.0564767
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
Better model found at epoch 7 with value: 1.0487560033798218.          
   epoch  train_loss  valid_loss    metric
0      7    1.592147    1.599561  1.048756
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
k

In [ ]:
k

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V8

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
graphnet: INFO     2023-02-07 13:16:52 - get_logger - Writing log to logs/graphnet_20230207-131652.log
Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_17', 'MODEL_WTS': '/opt/slh/icecube/RESULTS/EXP_16/EXP_16_9.pth', '__doc__': None}
Loading model weights from /opt/slh/icecube/RESULTS/EXP_16/EXP_16_9.pth
epoch     train_loss  valid_loss  val_metric
/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
wandb

In [ ]:
k